In [1]:


import os

os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:100"

import torch, sys
import json
import pandas as pd
import numpy as np


from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer
from transformers.trainer_pt_utils import LabelSmoother
from datasets import load_dataset, Dataset
from pprint import pprint

In [2]:

train_json_path = "/root/autodl-tmp/ie/yayi_uie_sft_data/train_sft.jsonl"

test_json_path = "/root/autodl-tmp/ie/yayi_uie_sft_data/test_sft.jsonl"




def load_dataset_from_disk(json_path):


    pass



dataset = load_dataset("json", data_files=test_json_path, split="train")
# print(dataset)
pprint(dataset[0])





{'Dataset': 'DuEE1.0',
 'Task': 'CEE',
 'input': '',
 'instruction': '文本: 男子以帮找工作名义骗钱被警方抓获 \n'
                "【事件抽取】事件类型列表=['财经/交易-出售/收购', '财经/交易-跌停', '财经/交易-加息', "
                "'财经/交易-降价', '财经/交易-降息', '财经/交易-融资', '财经/交易-上市', '财经/交易-涨价', "
                "'财经/交易-涨停', '产品行为-发布', '产品行为-获奖', '产品行为-上映', '产品行为-下架', "
                "'产品行为-召回', '交往-道歉', '交往-点赞', '交往-感谢', '交往-会见', '交往-探班', "
                "'竞赛行为-夺冠', '竞赛行为-晋级', '竞赛行为-禁赛', '竞赛行为-胜负', '竞赛行为-退赛', "
                "'竞赛行为-退役', '人生-产子/女', '人生-出轨', '人生-订婚', '人生-分手', '人生-怀孕', "
                "'人生-婚礼', '人生-结婚', '人生-离婚', '人生-庆生', '人生-求婚', '人生-失联', "
                "'人生-死亡', '司法行为-罚款', '司法行为-拘捕', '司法行为-举报', '司法行为-开庭', "
                "'司法行为-立案', '司法行为-起诉', '司法行为-入狱', '司法行为-约谈', '灾害/意外-爆炸', "
                "'灾害/意外-车祸', '灾害/意外-地震', '灾害/意外-洪灾', '灾害/意外-起火', '灾害/意外-坍/垮塌', "
                "'灾害/意外-袭击', '灾害/意外-坠机', '组织关系-裁员', '组织关系-辞/离职', '组织关系-加盟', "
                "'组织关系-解雇', '组织关系-解散', '组织关系-解约', '组织关系-停职', '组织关系-退出', "


In [3]:
"""初始化一些参数"""
# model_name = "Qwen/Qwen2-7B-Instruct"
# cache_dir = "/root/autodl-tmp/qwen2-7b-instruct"

model_name = "Qwen/Qwen1.5-7B-Chat"
cache_dir = "/root/autodl-tmp/Qwen1.5-7B-Chat/"
tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir=cache_dir, trust_remote_code=True, padding="right")
max_length = None
model = AutoModelForCausalLM.from_pretrained(model_name, cache_dir=cache_dir, device_map="auto", torch_dtype=torch.bfloat16)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [4]:
"""测试tokenizer"""

print(tokenizer("稀土开发者"), tokenizer.decode([113129]))

# one_example = dataset[0]
# print(type(one_example))
# pprint(one_example)


"""
测试对话模板
"""
prompt = ""

messages = [
    {"role": "assistant", "content": "You are a helpful assistant."},
    {"role": "user", "content": "我要退货"},
    {"role": "system", "content": "请提供单号"},
    {"role": "user", "content": "123456"},
    {"role": "assistant", "content": "已为您退货完成"}
]

text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

print(text)

print(tokenizer.chat_template)


{'input_ids': [113921, 113129], 'attention_mask': [1, 1]} 开发者
<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>assistant
You are a helpful assistant.<|im_end|>
<|im_start|>user
我要退货<|im_end|>
<|im_start|>system
请提供单号<|im_end|>
<|im_start|>user
123456<|im_end|>
<|im_start|>assistant
已为您退货完成<|im_end|>
<|im_start|>assistant

{% for message in messages %}{% if loop.first and messages[0]['role'] != 'system' %}{{ '<|im_start|>system
You are a helpful assistant.<|im_end|>
' }}{% endif %}{{'<|im_start|>' + message['role'] + '
' + message['content'] + '<|im_end|>' + '
'}}{% endfor %}{% if add_generation_prompt %}{{ '<|im_start|>assistant
' }}{% endif %}


In [5]:



# print(one_example["instruction"])


from typing import Dict, List

# 参考qwen2官方参考的finetuning代码，发现是错误的
def process_fun_error(example: Dict):
    """自己实现instruction和response的区分"""
    system_prompt = "你是一个文本信息抽取领域的专家，现在需要你从用户user的给定文本中任务类型完成对应的信息抽取任务"
    instruction = example["instruction"]
    response = example["label"]
    messages = [{"role": "user", "content": instruction}, {"role": "system", "content": labels}]
    tokenized_message_ids = tokenizer.apply_chat_template(messages, 
                                                      tokenize=True, 
                                                      chat_template=tokenizer.chat_template,
                                                      add_generation_prompt=False, 
                                                      padding="max_length", 
                                                      max_length=8192, 
                                                      truncation=True)  # # list[int]
    # print(tokenizer.decode(tokenized_message_ids))
    input_ids = torch.tensor(tokenized_message_ids, dtype=torch.int)
    target_ids = input_ids.clone()
    target_ids[target_ids==tokenizer.pad_token_id] = LabelSmoother.ignore_index  # -100
    attenion_mask = input_ids.ne(tokenizer.pad_token_id).int()   # boolean转int类型
    return dict(input_ids=input_ids, attenion_mask=attenion_mask, target_ids=target_ids)
    


def process_fun_qwen15(example):
    
    instruction = example["instruction"]
    response = example["label"]
    # messages = [{"role": "user", "content": instruction}, {"role": "system", "content": labels}]
    # tokenized_message_ids = tokenizer.apply_chat_template(messages, 
    #                                                   tokenize=True, 
    #                                                   chat_template=tokenizer.chat_template,
    #                                                   add_generation_prompt=False, 
    #                                                   padding="max_length", 
    #                                                   max_length=8192, 
    #                                                   truncation=True)  # # list[int]
    
    template = f"<|im_start|>system\n你是一个文本信息抽取领域的专家，现在需要你从用户user的给定文本中任务类型完成对应的信息抽取任务<|im_end|>\n<|im_start|>user\n{instruction }<|im_end|>\n<|im_start|>assistant\n"
    tokenized_instruction = tokenizer(template, add_special_tokens=False)
    tokenized_response = tokenizer(response, add_special_tokens=False)
    
    input_ids = tokenized_instruction["input_ids"] + tokenized_response["input_ids"] + [tokenizer.pad_token_id]
    attention_mask = tokenized_instruction["attention_mask"] + tokenized_response["attention_mask"] + [1]
    labels = [-100] * len(tokenized_instruction["input_ids"]) + tokenized_response["input_ids"] + [tokenizer.pad_token_id]

    if len(input_ids) > max_length:  # input_ids 已经包含了输入和输出的id
        input_ids = input_ids[:max_length]
        attention_mask = attention_mask[:max_length]
        labels = labels[:max_length]
    return {"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels}


# example_inputs = process_fun(one_example)
# tt = example_inputs["target_ids"]
# print("target_padding的大小：", tt.shape)
# print(tt,tokenizer.pad_token_id, LabelSmoother.ignore_index)

# ta = tt[tt != -100]
# print("target的大小：", ta.shape)
# # print(tokenizer.decode(ta))

    
# ii = example_inputs["input_ids"]
# print("解码如下--------------------------：\n", tokenizer.decode(ii))



tokenized_dataset = dataset.map(process_fun_qwen15, remove_columns=dataset.column_names, batched=False)




print(tokenized_dataset)


Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 5450
})


In [6]:

# 定义Lora


from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(task_type=TaskType.CAUSAL_LM, 
                         # target_modules=["q_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
                         target_modules=["q_proj", "k_proj", "v_proj"],
                         inference_mode=False,lora_alpha=32, r=8, lora_dropout=0.1)

# print("lora config: ", lora_config)
# gradient_checkpoint=True, 必须设置: model.enable_input_require_grads()

gradient_checkpointing = False
if gradient_checkpointing:
    model.enable_input_require_grads()

model = get_peft_model(model=model, peft_config=lora_config)
# print(model)
print("模型可训练参数：")
model.print_trainable_parameters()



args = TrainingArguments(output_dir="/root/autodl-tmp/yayi_output/",
                         per_device_eval_batch_size=1, gradient_accumulation_steps=1, 
                         logging_dir="/root/autodl-tmp/yayi_output/logging",
                         logging_steps=1, 
                         learning_rate=3e-5, 
                         save_steps=50, save_total_limit=1,
                         num_train_epochs=3, 
                         gradient_checkpointing=False)


trainer = Trainer(model=model, args=args, 
                  data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True, label_pad_token_id=-100),
                  train_dataset=tokenized_dataset)


trainer.train()



模型可训练参数：
trainable params: 6,291,456 || all params: 7,727,616,000 || trainable%: 0.0814
[2024-07-09 17:23:45,451] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-dev package with apt
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH
 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.1
 [WARNING]  using untested triton version (2.1.0), only 1.0.0 is known to be compatible


/root/miniconda3/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


OutOfMemoryError: CUDA out of memory. Tried to allocate 98.00 MiB. GPU 1 has a total capacty of 23.64 GiB of which 97.75 MiB is free. Process 802419 has 23.54 GiB memory in use. Of the allocated memory 22.57 GiB is allocated by PyTorch, and 580.60 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
# """
# 参考： https://github.com/QwenLM/Qwen2/blob/main/examples/sft/finetune.py#L144
# """

# from transformers import PreTrainedTokenizer
# import transformers

# TEMPLATE = "{% for message in messages %}{% if loop.first and messages[0]['role'] != 'system' %}{{ '<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n' }}{% endif %}{{'<|im_start|>' + message['role'] + '\n' + message['content']}}{% if loop.last %}{{ '<|im_end|>'}}{% else %}{{ '<|im_end|>\n' }}{% endif %}{% endfor %}"



# def preprocess(
#     messages,
#     tokenizer: transformers.PreTrainedTokenizer,
#     max_len: int,
# ) -> Dict:
#     """Preprocesses the data for supervised fine-tuning."""

#     texts = []
#     for i, msg in enumerate(messages):
#         texts.append(
#             tokenizer.apply_chat_template(
#                 msg,
#                 chat_template=TEMPLATE,
#                 tokenize=True,
#                 add_generation_prompt=False,
#                 padding="max_length",
#                 max_length=max_len,
#                 truncation=True,
#             )
#         )
#     input_ids = torch.tensor(texts, dtype=torch.int)
#     target_ids = input_ids.clone()
#     target_ids[target_ids == tokenizer.pad_token_id] = LabelSmoother.ignore_index
#     attention_mask = input_ids.ne(tokenizer.pad_token_id)

#     return dict(
#         input_ids=input_ids, target_ids=target_ids, attention_mask=attention_mask
#     )




# # pprint(one_example)

# one_messages = [{"role": "user", "content": one_example["instruction"]}, {"role": "system", "content": one_example["label"]}]
# # print(one_messages)


# ins = preprocess([one_messages], tokenizer, max_len=8192)

# ins_ids = ins["input_ids"][0]
# # print(tokenizer.decode(ins_ids[ins_ids!=151643151643]))
# print(ins["attention_mask"][0])
